#Dependencies

In [ ]:
!pip install evaluate

In [ ]:
! pip install -U accelerate
! pip install -U transformers sentencepiece

In [ ]:
!pip install Datasets

In [4]:
import numpy as np
import pandas as pd

from datasets import load_dataset,Dataset,DatasetDict,ClassLabel,Features
from transformers import DataCollatorWithPadding,AutoModelForTokenClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd


In [5]:
import re
import os
import json
import spacy
from spacy.training import biluo_tags_to_offsets, offsets_to_biluo_tags

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 950.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=6f6ace70016985a52616148ed03f2985847fd8c4e6264eb508b3c9ed388a861f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


#Download Dataset


In [8]:
!curl -O https://raw.githubusercontent.com/taisti/TASTEset-2.0/main/data/TASTEset.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1612k  100 1612k    0     0  3462k      0 --:--:-- --:--:-- --:--:-- 3460k


#Loading and Pre-Processing Data

In [9]:
ds = pd.read_csv("TASTEset.csv")

In [10]:
NLP = spacy.load('en_core_web_sm')
ENTITIES = ["FOOD", "QUANTITY", "UNIT", "PROCESS", "PHYSICAL_QUALITY", "COLOR",
            "TASTE", "PURPOSE", "PART"]


In [11]:
def prepare_data(taste_set, entities_format="spans", discontinuous=False):
    """
    :param tasteset: TASTEset as pd.DataFrame or a path to the TASTEset
    :param entities_format: the format of entities. If equal to 'bio', entities
    will be of the following format: [[B-FOOD, I-FOOD, O, ...], [B-UNIT, ...]].
    If equal to span, entities will be of the following format:
    [[(0, 6, FOOD), (10, 15, PROCESS), ...], [(0, 2, UNIT), ...]]
    :param discontinuous: if True, then include discontinuous entites
    :return: list of recipes ingredients and corresponding list of entities
    """
    """source : https://github.com/taisti/TASTEset-2.0/blob/adf9da58737c891559e779658a7ce2b98f805657/src/utils.py#L16"""

    assert entities_format in ["bio", "spans"],\
        'You provided incorrect entities format!'
    if isinstance(taste_set, pd.DataFrame):
        df = taste_set
    elif isinstance(taste_set, str) and os.path.exists(taste_set):
        df = pd.read_csv(taste_set)
    else:
        raise ValueError('Incorret TASTEset format!')

    all_ingredients = df["ingredients"].to_list()
    all_entities = []

    if discontinuous:
        raise NotImplementedError("The model does not handle discontinuity!")

    for idx in df.index:
        ingredients_entities = json.loads(df.at[idx, "ingredients_entities"])
        entities = []

        for entity_dict in ingredients_entities:
            # pick only specified entities
            if entity_dict["type"] not in ENTITIES:
                continue
            spans = entity_dict["span"]
            spans = re.findall("(\d+, \d+)", spans)
            spans = [[int(char_id) for char_id in span.split(",")] for span
                     in spans]
            for start, end in spans:
                add = True
                # avoid overlapping entities
                for present_start, present_end, _ in entities:
                    if start >= present_start and end <= present_end:
                        add = False
                if add:
                    entities.append((start, end, entity_dict["type"]))

        if entities_format == "bio":
            tokenized_ingredients, entities = span_to_bio(all_ingredients[idx],
                                                          entities)
            tokenized_ingredients = [NEWLINE_CHAR if token == "\n" else token
                    for token in tokenized_ingredients]
            all_ingredients[idx] = tokenized_ingredients

        all_entities.append(entities)

    return all_ingredients, all_entities

In [12]:
def span_to_bio(recipe, span_entities):
    """
    :param span_entities: list of span entities, eg. [(span_start, span_end,
    "FOOD"), (span_start, span_end, "PROCESS")]
    :return: list of BIO entities, eg. ["O", "B-FOOD", "I-FOOD", "B-PROCESS"]
    """
    """source : https://github.com/taisti/TASTEset-2.0/blob/adf9da58737c891559e779658a7ce2b98f805657/src/utils.py#L16"""


    tokenized_recipe, biluo_entities = span_to_biluo(recipe, span_entities)
    bio_entities = biluo_to_bio(biluo_entities)
    return tokenized_recipe, bio_entities

In [13]:
def span_to_biluo(recipe, span_entities):
    """
    :param span_entities: list of span entities, eg. [(span_start, span_end,
    "FOOD"), (span_start, span_end, "PROCESS")]
    :return: list of BILUO entities, eg. ["O", "B-FOOD", "L-FOOD",
    "U-PROCESS"] along with tokenized recipe
    """
    """source : https://github.com/taisti/TASTEset-2.0/blob/adf9da58737c891559e779658a7ce2b98f805657/src/utils.py#L16"""

    doc = NLP(recipe.replace("\n", " "))
    tokenized_recipe = [token.text for token in doc]
    spans = offsets_to_biluo_tags(doc, span_entities)
    return tokenized_recipe, spans

In [14]:
def biluo_to_bio(biluo_entities):
    """
    :param biluo_entities: list of BILUO entities, eg. ["O", "B-FOOD", "L-FOOD",
    "U-PROCESS"]
    :return: list of BIO entities, eg. ["O", "B-FOOD", "I-FOOD", "B-PROCESS"]
    """
    """source : https://github.com/taisti/TASTEset-2.0/blob/adf9da58737c891559e779658a7ce2b98f805657/src/utils.py#L16"""

    bio_entities = [entity.replace("L-", "I-").replace("U-", "B-")
                    for entity in biluo_entities]
    return bio_entities

In [15]:
tokens, ner_tags = prepare_data(ds,entities_format="bio")

In [16]:
pd.DataFrame([tokens[0],ner_tags[0]]) # view tokens and corresponding ner_tags for a pair

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,5,ounces,rum,4,ounces,triple,sec,3,ounces,Tia,Maria,20,ounces,orange,juice
1,B-QUANTITY,B-UNIT,B-FOOD,B-QUANTITY,B-UNIT,B-FOOD,I-FOOD,B-QUANTITY,B-UNIT,O,O,B-QUANTITY,B-UNIT,B-FOOD,I-FOOD


In [17]:
flat_list = [item for sublist in ner_tags for item in sublist]

In [18]:
names = list(set(flat_list))
len(names)

19

In [19]:
data_dict = {"tokens":tokens,"ner_tags":ner_tags} #creating a dict of all tokens and corresponding ner_tags


In [20]:
from datasets import Features,Value,Sequence

features = Features({
    "tokens": Sequence(Value(dtype = "string")),
    "ner_tags": Sequence(ClassLabel(num_classes=19,names=names))
})

features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['I-PHYSICAL_QUALITY', 'B-TASTE', 'I-UNIT', 'I-PROCESS', 'B-PART', 'I-PURPOSE', 'B-PURPOSE', 'B-QUANTITY', 'B-UNIT', 'B-PHYSICAL_QUALITY', 'B-FOOD', 'I-COLOR', 'I-QUANTITY', 'I-PART', 'O', 'I-TASTE', 'B-PROCESS', 'I-FOOD', 'B-COLOR'], id=None), length=-1, id=None)}

In [21]:
dataset = Dataset.from_dict(data_dict,features=features)

In [22]:
dataset[0]

{'tokens': ['5',
  'ounces',
  'rum',
  '4',
  'ounces',
  'triple',
  'sec',
  '3',
  'ounces',
  'Tia',
  'Maria',
  '20',
  'ounces',
  'orange',
  'juice'],
 'ner_tags': [7, 8, 10, 7, 8, 10, 17, 7, 8, 14, 14, 7, 8, 10, 17]}

In [23]:
class_names = dataset.features['ner_tags'].feature.names
#class_names

In [24]:
def map_ner_tags_to_names(example):
    example['ner_tags_str'] = [class_names[tag] for tag in example['ner_tags']]
    return example

In [25]:
dataset = dataset.map(map_ner_tags_to_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [26]:
sample = dataset[0]
pd.DataFrame([sample['tokens'],sample['ner_tags'],sample['ner_tags_str']],index = ['tokens','ner_tags','ner_tags_str'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
tokens,5,ounces,rum,4,ounces,triple,sec,3,ounces,Tia,Maria,20,ounces,orange,juice
ner_tags,7,8,10,7,8,10,17,7,8,14,14,7,8,10,17
ner_tags_str,B-QUANTITY,B-UNIT,B-FOOD,B-QUANTITY,B-UNIT,B-FOOD,I-FOOD,B-QUANTITY,B-UNIT,O,O,B-QUANTITY,B-UNIT,B-FOOD,I-FOOD


In [27]:
dataset = dataset.train_test_split(test_size=0.2)

In [28]:
dataset['train'] = dataset['train'].train_test_split(test_size=0.25)

In [29]:
dataset

DatasetDict({
    train: DatasetDict({
        train: Dataset({
            features: ['tokens', 'ner_tags', 'ner_tags_str'],
            num_rows: 600
        })
        test: Dataset({
            features: ['tokens', 'ner_tags', 'ner_tags_str'],
            num_rows: 200
        })
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 200
    })
})

In [30]:
train = dataset['train']['train']
validation = dataset['train']['test']
test = dataset['test']

In [31]:
new_ds = DatasetDict({
    'train':train,
    'validation':validation,
    'test':test
})

In [32]:
new_ds

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 600
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 200
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 200
    })
})

#tokenization

In [33]:
model_nm = 'microsoft/deberta-v3-small' #model_name

In [34]:
tokz = AutoTokenizer.from_pretrained(model_nm) #tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [35]:
example_text = new_ds['train']['tokens'][19]
labels = new_ds['train']['ner_tags'][19]
labels_str = new_ds['train']['ner_tags_str'][19]

In [36]:
tokenized_example_text = tokz(example_text, is_split_into_words=True)

In [37]:
word_ids = tokenized_example_text.word_ids()

In [38]:
pd.DataFrame([example_text,tokenized_example_text.tokens(),labels_str, tokenized_example_text.word_ids()])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,4,cups,flour,1/2,teaspoon,salt,1,teaspoon,baking,soda,1,3/4,cups,milk,None,None,None,None,None,None
1,[CLS],▁4,▁cups,▁flour,▁1,/,2,▁teaspoon,▁salt,▁1,▁teaspoon,▁baking,▁soda,▁1,▁3,/,4,▁cups,▁milk,[SEP]
2,B-QUANTITY,B-UNIT,B-FOOD,B-QUANTITY,B-UNIT,B-FOOD,B-QUANTITY,B-UNIT,B-FOOD,I-FOOD,B-QUANTITY,I-QUANTITY,B-UNIT,B-FOOD,None,None,None,None,None,None
3,None,0,1,2,3,3,3,4,5,6,7,8,9,10,11,11,11,12,13,None


In [39]:
previous_word_idx = None
label_ids = []

for word_idx in word_ids:
    if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100)
    elif word_idx != previous_word_idx:
        label_ids.append(labels[word_idx])
    previous_word_idx = word_idx

labels = [class_names[l] if l != -100 else "IGN" for l in label_ids]
index = ["Tokens", "Word IDs", "Label IDs", "Labels"]

pd.DataFrame([tokenized_example_text.tokens(), word_ids, label_ids, labels], index=index)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,[CLS],▁4,▁cups,▁flour,▁1,/,2,▁teaspoon,▁salt,▁1,▁teaspoon,▁baking,▁soda,▁1,▁3,/,4,▁cups,▁milk,[SEP]
Word IDs,None,0,1,2,3,3,3,4,5,6,7,8,9,10,11,11,11,12,13,None
Label IDs,-100,7,8,10,7,-100,-100,8,10,7,8,10,17,7,12,-100,-100,8,10,-100
Labels,IGN,B-QUANTITY,B-UNIT,B-FOOD,B-QUANTITY,IGN,IGN,B-UNIT,B-FOOD,B-QUANTITY,B-UNIT,B-FOOD,I-FOOD,B-QUANTITY,I-QUANTITY,IGN,IGN,B-UNIT,B-FOOD,IGN


In [40]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokz(examples["tokens"],is_split_into_words = True,padding=True)

    labels = []
    #import pdb; pdb.set_trace()
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None or word_idx == previous_word_idx:
            label_ids.append(-100)
        else:
            label_ids.append(examples['ner_tags'][word_idx])
        previous_word_idx = word_idx
        #import pdb; pdb.set_trace()
    labels.append(label_ids)
    #import pdb; pdb.set_trace()
    tokenized_inputs["labels"] = label_ids

    return tokenized_inputs

In [41]:
train_tok_ds = new_ds['train'].map(tokenize_and_align_labels,remove_columns= ['tokens','ner_tags','ner_tags_str'])

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [42]:
train_tok_ds['labels'][0]

[-100,
 7,
 8,
 16,
 10,
 7,
 8,
 10,
 17,
 7,
 8,
 10,
 7,
 8,
 10,
 10,
 10,
 7,
 12,
 -100,
 8,
 10,
 17,
 14,
 7,
 12,
 -100,
 8,
 10,
 -100]

In [43]:
val_tok_ds = new_ds['validation'].map(tokenize_and_align_labels,remove_columns= ['tokens','ner_tags','ner_tags_str'])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [44]:
eval_tok_ds = new_ds['test'].map(tokenize_and_align_labels,remove_columns= ['tokens','ner_tags','ner_tags_str'])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [45]:
eval_tok_ds

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 200
})

In [46]:
pd.DataFrame(eval_tok_ds[:5])

,input_ids,token_type_ids,attention_mask,labels
0,"[1, 392, 4167, 404, 2423, 14413, 287, 273, 427...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 7, 10, 7, 8, 10, 14, 14, 14, 9, 10, 14,..."
1,"[1, 621, 7048, 287, 536, 353, 8642, 1263, 392,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 7, 10, 14, 9, 9, 16, 14, 7, 8, 10, 17, ..."
2,"[1, 525, 7634, 986, 1818, 287, 273, 1021, 291,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 7, 8, 1, 10, 14, 14, 14, 14, 14, 14, 14..."
3,"[1, 392, 4167, 287, 288, 629, 2247, 1263, 3194...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 7, 10, 14, 14, 9, 0, 14, 10, 10, 7, 8, ..."
4,"[1, 376, 406, 597, 44033, 28070, 3189, 3987, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 9, 0, 0, 10, 7, 8, 10, 14, 16, 7, 8, 10..."


#model

In [47]:
index2tag = {idx: tag for idx, tag in enumerate(class_names)}
tag2index = {tag: idx for idx, tag in enumerate(class_names)}

In [48]:
model = AutoModelForTokenClassification.from_pretrained(model_nm,num_labels=19,id2label=index2tag, label2id=tag2index)


pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [51]:
a = new_ds['train']['tokens'][0]

In [52]:
b = tokz.encode(a,return_tensors='pt',is_split_into_words=True)

In [53]:
outputs = model(b).logits
predictions = outputs.argmax(dim=-1)
predictions = predictions.tolist()
outputs.shape

torch.Size([1, 30, 19])

# Metrics


In [132]:
def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100 :
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)
    #print(preds_list[0]), print(labels_list[0])
    return preds_list, labels_list

In [133]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions,
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

In [ ]:
eval_tok_ds['labels'][0]

#Model


In [135]:
from transformers import DataCollatorForTokenClassification

In [136]:
data_collator = DataCollatorForTokenClassification(tokz)

In [137]:
bs = 20
epochs = 5
lr = 8e-5

In [138]:
def model_init():
  return  (AutoModelForTokenClassification
           .from_pretrained(model_nm,num_labels=19,id2label=index2tag, label2id=tag2index)
           .to(device))

In [139]:
args = TrainingArguments('outputs', learning_rate=lr,log_level= "error", warmup_ratio=0.1, lr_scheduler_type='cosine',
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,disable_tqdm =False,
    num_train_epochs=epochs, weight_decay=0.01,logging_steps = 30)

In [140]:
trainer = Trainer(model_init=model_init, args=args,
                  data_collator = data_collator,
                  train_dataset=train_tok_ds,
                  compute_metrics= compute_metrics,
                  eval_dataset=val_tok_ds,
                  tokenizer=tokz)

In [141]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,1.296500,0.391147,0.884058
2,0.322600,0.264133,0.923964
3,0.192800,0.236217,0.937475
4,0.131200,0.239297,0.942002
5,0.113100,0.235524,0.939778


TrainOutput(global_step=150, training_loss=0.41125411828358965, metrics={'train_runtime': 35.2248, 'train_samples_per_second': 85.167, 'train_steps_per_second': 4.258, 'total_flos': 54734884045800.0, 'train_loss': 0.41125411828358965, 'epoch': 5.0})

In [142]:
trainer.evaluate()

{'eval_loss': 0.23552381992340088,
 'eval_f1': 0.9397781299524565,
 'eval_runtime': 0.6788,
 'eval_samples_per_second': 294.631,
 'eval_steps_per_second': 7.366,
 'epoch': 5.0}

#Test

In [143]:
preds = trainer.predict(eval_tok_ds)
preds.metrics

{'test_loss': 0.2584766149520874,
 'test_f1': 0.9381685363944692,
 'test_runtime': 0.6434,
 'test_samples_per_second': 310.836,
 'test_steps_per_second': 7.771}

In [72]:
predictions, labels, _ = trainer.predict(eval_tok_ds)
predictions = np.argmax(predictions,axis = 2)

In [ ]:
predictions[0],labels[0]

In [ ]:
class_names

In [75]:
true_predictions = [
    [class_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [class_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

In [ ]:
from datasets import load_metric
metric = load_metric("seqeval")

In [152]:
results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'COLOR': {'precision': 0.9,
  'recall': 0.9402985074626866,
  'f1': 0.9197080291970803,
  'number': 67},
 'FOOD': {'precision': 0.9065502183406113,
  'recall': 0.9218472468916519,
  'f1': 0.9141347424042272,
  'number': 1126},
 'PART': {'precision': 0.75,
  'recall': 0.6521739130434783,
  'f1': 0.6976744186046512,
  'number': 23},
 'PHYSICAL_QUALITY': {'precision': 0.75,
  'recall': 0.8325991189427313,
  'f1': 0.7891440501043842,
  'number': 227},
 'PROCESS': {'precision': 0.889937106918239,
  'recall': 0.9433333333333334,
  'f1': 0.9158576051779936,
  'number': 300},
 'PURPOSE': {'precision': 0.75,
  'recall': 0.7241379310344828,
  'f1': 0.736842105263158,
  'number': 29},
 'QUANTITY': {'precision': 0.9772105742935278,
  'recall': 0.988929889298893,
  'f1': 0.9830353049060064,
  'number': 1084},
 'TASTE': {'precision': 0.7037037037037037,
  'recall': 0.6551724137931034,
  'f1': 0.6785714285714286,
  'number': 29},
 'UNIT': {'precision': 0.9824561403508771,
  'recall': 0.9824561403508